# ✅ Bank System by Jo Foundation
Final Project — Google Colab UI Version


This notebook contains:

✅ Bank System Module Import  
✅ Interactive UI for account management  
✅ Simple functionality test  



### 📌 Step 1 — Upload `bank_system.py` file
Before running the UI, upload the `bank_system.py` file you downloaded earlier.

You can upload using the file panel on the left or run:
```python
from google.colab import files
uploaded = files.upload()
```


In [ ]:
# ✅ Step 2 — Load module and setup UI

"""
Bank System by Jo Foundation
Core module for Phase 1: multi-account bank system with username + PIN (4 digits)
Features:
- Create account (username, 4-digit PIN, initial deposit)
- Login / logout
- Deposit
- Withdraw (with balance checks)
- Check balance
- Transaction history per account
- Persistence to JSON (accounts.json)
Security:
- PINs are stored as SHA-256 hashes (not plain text) for minimal security awareness.
Usage:
- Import this module in Google Colab or run it as a script.
- A simple CLI demo is included under `if __name__ == '__main__'`.
"""

import json
import os
from dataclasses import dataclass, asdict, field
from typing import Dict, List, Optional
import hashlib
import datetime

ACCOUNTS_FILE = '/mnt/data/accounts.json'  # In Colab this path will persist for the session only


def hash_pin(pin: str) -> str:
    """Return SHA-256 hash of the PIN string."""
    return hashlib.sha256(pin.encode('utf-8')).hexdigest()


def now_iso():
    return datetime.datetime.utcnow().isoformat() + 'Z'


@dataclass
class Transaction:
    timestamp: str
    type: str  # "deposit" or "withdraw"
    amount: float
    balance_after: float
    note: Optional[str] = None


@dataclass
class BankAccount:
    username: str
    pin_hash: str
    balance: float = 0.0
    transactions: List[Dict] = field(default_factory=list)

    def check_pin(self, pin: str) -> bool:
        return self.pin_hash == hash_pin(pin)

    def deposit(self, amount: float, note: Optional[str] = None) -> Transaction:
        if amount <= 0:
            raise ValueError("Deposit amount must be positive.")
        self.balance += amount
        tx = Transaction(timestamp=now_iso(), type="deposit", amount=amount, balance_after=self.balance, note=note)
        self.transactions.append(asdict(tx))
        return tx

    def withdraw(self, amount: float, note: Optional[str] = None) -> Transaction:
        if amount <= 0:
            raise ValueError("Withdraw amount must be positive.")
        if amount > self.balance:
            raise ValueError("Insufficient funds.")
        self.balance -= amount
        tx = Transaction(timestamp=now_iso(), type="withdraw", amount=amount, balance_after=self.balance, note=note)
        self.transactions.append(asdict(tx))
        return tx

    def get_balance(self) -> float:
        return self.balance

    def get_statement(self, n: int = 10) -> List[Dict]:
        return list(reversed(self.transactions))[:n]


class AccountManager:
    def __init__(self, storage_file: str = ACCOUNTS_FILE):
        self.storage_file = storage_file
        self.accounts: Dict[str, BankAccount] = {}
        self.logged_in: Optional[BankAccount] = None
        self.load()

    def load(self):
        if os.path.exists(self.storage_file):
            try:
                with open(self.storage_file, 'r') as f:
                    data = json.load(f)
                for username, info in data.items():
                    acc = BankAccount(username=username, pin_hash=info['pin_hash'], balance=info.get('balance', 0.0), transactions=info.get('transactions', []))
                    self.accounts[username] = acc
            except Exception as e:
                print(f"[WARN] Failed to load accounts from {self.storage_file}: {e}")

    def save(self):
        data = {}
        for username, acc in self.accounts.items():
            data[username] = {
                'pin_hash': acc.pin_hash,
                'balance': acc.balance,
                'transactions': acc.transactions
            }
        os.makedirs(os.path.dirname(self.storage_file), exist_ok=True)
        with open(self.storage_file, 'w') as f:
            json.dump(data, f, indent=2)
        return self.storage_file

    def create_account(self, username: str, pin: str, initial_deposit: float = 0.0) -> BankAccount:
        if username in self.accounts:
            raise ValueError("Username already exists.")
        if not (pin.isdigit() and len(pin) == 4):
            raise ValueError("PIN must be 4 digits.")
        if initial_deposit < 0:
            raise ValueError("Initial deposit cannot be negative.")
        acc = BankAccount(username=username, pin_hash=hash_pin(pin), balance=float(initial_deposit))
        if initial_deposit > 0:
            # record initial deposit transaction
            acc.transactions.append(asdict(Transaction(timestamp=now_iso(), type="deposit", amount=initial_deposit, balance_after=acc.balance, note="initial_deposit")))
        self.accounts[username] = acc
        self.save()
        return acc

    def login(self, username: str, pin: str) -> BankAccount:
        acc = self.accounts.get(username)
        if acc is None:
            raise ValueError("Account not found.")
        if not acc.check_pin(pin):
            raise ValueError("Incorrect PIN.")
        self.logged_in = acc
        return acc

    def logout(self):
        self.logged_in = None

    def deposit_logged(self, amount: float, note: Optional[str] = None) -> Transaction:
        if not self.logged_in:
            raise RuntimeError("No user logged in.")
        tx = self.logged_in.deposit(amount, note=note)
        self.save()
        return tx

    def withdraw_logged(self, amount: float, note: Optional[str] = None) -> Transaction:
        if not self.logged_in:
            raise RuntimeError("No user logged in.")
        tx = self.logged_in.withdraw(amount, note=note)
        self.save()
        return tx

    def get_balance_logged(self) -> float:
        if not self.logged_in:
            raise RuntimeError("No user logged in.")
        return self.logged_in.get_balance()

    def get_statement_logged(self, n: int = 10) -> List[Dict]:
        if not self.logged_in:
            raise RuntimeError("No user logged in.")
        return self.logged_in.get_statement(n=n)


# Simple CLI demo (useful when run as script or in a cell)
def cli_demo():
    mgr = AccountManager()
    print("=== Bank System by Jo Foundation — CLI Demo ===")
    while True:
        print("\\nOptions: [1] Create [2] Login [3] Exit")
        cmd = input("Choose: ").strip()
        if cmd == '1':
            uname = input("Choose username: ").strip()
            pin = input("Choose 4-digit PIN: ").strip()
            dep = float(input("Initial deposit (0 if none): ").strip() or 0)
            try:
                mgr.create_account(uname, pin, dep)
                print("Account created.")
            except Exception as e:
                print("Error:", e)
        elif cmd == '2':
            uname = input("Username: ").strip()
            pin = input("PIN: ").strip()
            try:
                acc = mgr.login(uname, pin)
                print(f\"Logged in as {acc.username}. Balance: {acc.balance}\")
                while True:
                    print(\"\\n[1] Balance  [2] Deposit [3] Withdraw [4] Statement [5] Logout\")
                    c2 = input(\"Choose: \").strip()
                    if c2 == '1':
                        print(\"Balance:\", mgr.get_balance_logged())
                    elif c2 == '2':
                        amt = float(input(\"Amount to deposit: \").strip())
                        mgr.deposit_logged(amt)
                        print(\"Deposited.\", \"Balance:\", mgr.get_balance_logged())
                    elif c2 == '3':
                        amt = float(input(\"Amount to withdraw: \").strip())
                        try:
                            mgr.withdraw_logged(amt)
                            print(\"Withdrawn.\", \"Balance:\", mgr.get_balance_logged())
                        except Exception as e:
                            print(\"Error:\", e)
                    elif c2 == '4':
                        for tx in mgr.get_statement_logged(20):
                            print(tx)
                    elif c2 == '5':
                        mgr.logout()
                        print(\"Logged out.\")
                        break
                    else:
                        print(\"Unknown option.\")
            except Exception as e:
                print(\"Login failed:\", e)
        elif cmd == '3':
            print(\"Goodbye.\")
            break
        else:
            print(\"Unknown option.\")


if __name__ == '__main__':
    cli_demo()


In [ ]:

# ✅ Step 3 — Launch UI in Colab

# !pip install ipywidgets  # uncomment if needed

from bank_system import AccountManager
import ipywidgets as widgets
from IPython.display import display, clear_output

mgr = AccountManager('/content/accounts.json')  # persistent inside session

out = widgets.Output()
display(out)

tab = widgets.Tab()

uname_create = widgets.Text(description='Username:')
pin_create = widgets.Password(description='PIN:')
dep_create = widgets.FloatText(description='Initial:')
btn_create = widgets.Button(description='Create Account')

uname_login = widgets.Text(description='Username:')
pin_login = widgets.Password(description='PIN:')
btn_login = widgets.Button(description='Login')

lbl_current = widgets.Label(value='Not logged in')
btn_balance = widgets.Button(description='Balance')
btn_deposit = widgets.Button(description='Deposit')
btn_withdraw = widgets.Button(description='Withdraw')
amt_field = widgets.FloatText(description='Amount:')
btn_logout = widgets.Button(description='Logout')
statement_area = widgets.Textarea(layout=widgets.Layout(width='100%', height='200px'))

create_box = widgets.VBox([uname_create, pin_create, dep_create, btn_create])
login_box = widgets.VBox([uname_login, pin_login, btn_login])
account_box = widgets.VBox([lbl_current, widgets.HBox([amt_field, btn_deposit, btn_withdraw]), btn_balance, btn_logout, statement_area])

tab.children = [create_box, login_box, account_box]
tab.set_title(0, 'Create')
tab.set_title(1, 'Login')
tab.set_title(2, 'Account')

def refresh_account_info():
    if mgr.logged_in:
        lbl_current.value = f'Logged in: {mgr.logged_in.username} | Balance: {mgr.get_balance_logged():.2f}'
        stmt = mgr.get_statement_logged(20)
        stmt_lines = [f"{t['timestamp']} | {t['type']} | {t['amount']} | bal: {t['balance_after']}" for t in stmt]
        statement_area.value = "\\n".join(stmt_lines)
    else:
        lbl_current.value = 'Not logged in'
        statement_area.value = ''

@out.capture()
def on_create(b):
    try:
        mgr.create_account(uname_create.value.strip(), pin_create.value.strip(), float(dep_create.value or 0.0))
        print(f"Account '{uname_create.value}' created!")
        uname_create.value = ''
        pin_create.value = ''
        dep_create.value = 0.0
        mgr.save()
    except Exception as e:
        print("Create failed:", e)

@out.capture()
def on_login(b):
    try:
        mgr.login(uname_login.value.strip(), pin_login.value.strip())
        print('Login successful ✅')
        uname_login.value = ''
        pin_login.value = ''
        refresh_account_info()
    except Exception as e:
        print('Login failed:', e)

@out.capture()
def on_deposit(b):
    try:
        mgr.deposit_logged(float(amt_field.value), note='manual_deposit')
        print('Deposit OK ✅')
        refresh_account_info()
    except Exception as e:
        print('Deposit failed:', e)

@out.capture()
def on_withdraw(b):
    try:
        mgr.withdraw_logged(float(amt_field.value), note='manual_withdraw')
        print('Withdraw OK ✅')
        refresh_account_info()
    except Exception as e:
        print('Withdraw failed:', e)

@out.capture()
def on_balance(b):
    try:
        bal = mgr.get_balance_logged()
        print(f'Balance: {bal:.2f} ✅')
    except Exception as e:
        print('Error:', e)

@out.capture()
def on_logout(b):
    mgr.logout()
    print('Logged out ✅')
    refresh_account_info()

btn_create.on_click(on_create)
btn_login.on_click(on_login)
btn_deposit.on_click(on_deposit)
btn_withdraw.on_click(on_withdraw)
btn_balance.on_click(on_balance)
btn_logout.on_click(on_logout)

refresh_account_info()
display(tab)


In [ ]:

# ✅ Step 4 — Basic Test Example (Auto Test)
from bank_system import AccountManager

print("Running basic tests...")

mgr_test = AccountManager('/content/test_accounts.json')

# Test: Create account
try:
    acc = mgr_test.create_account("tester", "1234", 50)
    print("✔ Create account: PASSED")
except Exception as e:
    print("✘ Create failed:", e)

# Test: Login
try:
    mgr_test.login("tester", "1234")
    print("✔ Login: PASSED")
except Exception as e:
    print("✘ Login failed:", e)

# Test: Deposit
try:
    mgr_test.deposit_logged(25)
    print("✔ Deposit: PASSED | Balance =", mgr_test.get_balance_logged())
except Exception as e:
    print("✘ Deposit failed:", e)

# Test: Withdraw
try:
    mgr_test.withdraw_logged(10)
    print("✔ Withdraw: PASSED | Balance =", mgr_test.get_balance_logged())
except Exception as e:
    print("✘ Withdraw failed:", e)

print("✅ Basic test complete!")
